In [160]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
import numpy as np
import pandas 
import matplotlib.pyplot as plt
import re
import os

import nltk
from nltk.tokenize import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords

from pymystem3 import Mystem


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split



russian_stopwords = stopwords.words("russian")
path = "C:\\Proganiy\\pp-laba3\\annotation.csv"
print(russian_stopwords)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [161]:
def create_ann(annatation: str) -> pandas.DataFrame:

    """Создаёт датафрейм по пути аннатации"""

    frame = pandas.DataFrame(columns =["Оценка","Kоличество слов","Текст рецензии"])
    ann_temp = open(annatation, "r", encoding="utf-8")
    for otzv in ann_temp.readlines():
        mas_otzv = otzv.split(",")
        otzv_temp = open(mas_otzv[0],"r",encoding="utf-8")
        otzv_text = " ".join(otzv_temp)
        row = pandas.Series({"Оценка": int(mas_otzv[2])-1,"Kоличество слов": len(otzv_text), "Текст рецензии": otzv_text})
        new_row = pandas.DataFrame([row], columns=frame.columns)
        frame = pandas.concat([frame, new_row], ignore_index=True)
    frame.dropna()
    return frame

data = create_ann(path)
print(data)

     Оценка Kоличество слов                                     Текст рецензии
0         0            1853  11 марта этого года переводила 145 в Германию(...
1         0            4278  В пятницу, 22 апреля 2022 г. Сбербанк заблокир...
2         0            2035  Здравствуйте. Перед оформлением досудебной пре...
3         0            3503  Я являюсь клиентом Сбербанка уже более десяти ...
4         0            3340  Призываю предпринимателей подумать дважды, пре...
...     ...             ...                                                ...
2995      4             304  Сегодня хочу рассказать о Сбербанке. В этом пр...
2996      4             863  Банк Сбербанк - это прекрасное финансовое учре...
2997      4             979  Пользуюсь сбербанком сколько себя помню, зарпл...
2998      4            2084  Я не сразу научилась пользоваться кредитной сб...
2999      4             777  Пользуюсь сбербанком давно, никаких нарекании ...

[3000 rows x 3 columns]


In [162]:
def clean (frame: pandas.DataFrame) -> pandas.DataFrame:
    frame["Текст рецензии"] = frame["Текст рецензии"].apply(delete_punc)
    print(frame)
    frame["Текст рецензии"] = frame["Текст рецензии"].apply(Mystem().lemmatize)
    print(frame)
    frame["Текст рецензии"] = frame["Текст рецензии"].apply(lambda x: ' '.join(x))
    print(frame)
    frame["Текст рецензии"] = frame["Текст рецензии"].apply(word_tokenize)
    print(frame)
    frame["Текст рецензии"] = frame["Текст рецензии"].apply(lambda x: ' '.join(x))
    return frame

def delete_punc(strings: str) -> str:
    punc = '''!()-[]{};:'",<>./?@#$%^&*~1234567890'''
    for char in strings:
        if char in punc:
            strings = strings.replace(char, "")
    return strings

data = clean(data)
print(data)

     Оценка Kоличество слов                                     Текст рецензии
0         0            1853   марта этого года переводила  в Германиюденежн...
1         0            4278  В пятницу  апреля  г Сбербанк заблокировал мою...
2         0            2035  Здравствуйте Перед оформлением досудебной прет...
3         0            3503  Я являюсь клиентом Сбербанка уже более десяти ...
4         0            3340  Призываю предпринимателей подумать дважды преж...
...     ...             ...                                                ...
2995      4             304  Сегодня хочу рассказать о Сбербанке В этом при...
2996      4             863  Банк Сбербанк  это прекрасное финансовое учреж...
2997      4             979  Пользуюсь сбербанком сколько себя помню зарпла...
2998      4            2084  Я не сразу научилась пользоваться кредитной сб...
2999      4             777  Пользуюсь сбербанком давно никаких нарекании н...

[3000 rows x 3 columns]
     Оценка Kоличество слов

In [206]:

max_words = 10000
cv = CountVectorizer(max_features=max_words)
sparse_matrix = cv.fit_transform(data['Текст рецензии']).toarray()
sparse_matrix.shape

print(type(np.array(data['Оценка'])))

<class 'numpy.ndarray'>


In [207]:
data['Оценка'] = data['Оценка'].astype (str).astype(int)
a=np.array(data['Оценка'])
x_train, x_, y_train, y_ = train_test_split(sparse_matrix, a, test_size=0.8)
x_test, x, y_test, y = train_test_split(sparse_matrix, a, test_size=0.2)
unique, counts = np.unique(y_train, return_counts=True)
print(dict(zip(unique, counts)))
x_train = Variable(torch.from_numpy(x_train)).float()
y_train = Variable(torch.from_numpy(y_train)).long()
x_test = Variable(torch.from_numpy(x_test)).float()
y_test = Variable(torch.from_numpy(y_test)).long()

{0: 117, 1: 116, 2: 116, 3: 117, 4: 134}


In [208]:


class LogisticRegression(nn.Module):
    def __init__(self):
        super(LogisticRegression, self).__init__()
        self.linear1 = nn.Linear(10000, 100)
        self.linear2 = nn.Linear(100, 20)
        self.linear3 = nn.Linear(20, 5)
        
    def forward(self, x):
        x = F.relu(self.linear1(x))
        x = F.relu(self.linear2(x))
        x = self.linear3(x)
        return F.softmax(x, dim=1)

model = LogisticRegression()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters() , lr=0.001)




In [209]:

epochs = 9
model.train()
loss_values = []
for epoch in range(epochs):
    optimizer.zero_grad()
    y_pred = model(x_train)
    loss = criterion(y_pred, y_train)
    loss_values.append(loss.item())
    pred = torch.max(y_pred, 1)[1].eq(y_train).sum()
    acc = pred * 100.0 / len(x_train)
    print('Epoch: {}, Accuracy: {}%'.format(epoch+1, acc.numpy()))
    loss.backward()
    optimizer.step()

Epoch: 1, Accuracy: 22.5%
Epoch: 2, Accuracy: 27.33333396911621%
Epoch: 3, Accuracy: 31.16666603088379%
Epoch: 4, Accuracy: 35.33333206176758%
Epoch: 5, Accuracy: 37.83333206176758%
Epoch: 6, Accuracy: 41.16666793823242%
Epoch: 7, Accuracy: 45.33333206176758%
Epoch: 8, Accuracy: 48.33333206176758%
Epoch: 9, Accuracy: 53.0%


In [210]:


model.eval()
with torch.no_grad():
    y_pred = model(x_test)
    loss = criterion(y_pred, y_test)
    pred = torch.max(y_pred, 1)[1].eq(y_test).sum()
    print ("Accuracy : {}%".format(100*pred/len(x_test)))

Accuracy : 33.375%
